In [1]:
from xml.etree import ElementTree
import re
import numpy as np
from numpy.typing import NDArray
import pandas as pd
from sympy.geometry import Plane, Point3D, Segment3D
from typing import Any, Dict, List, Union, Optional, Tuple, Iterable
from itertools import islice
from dataclasses import dataclass
from __future__ import annotations

In [2]:
tree = ElementTree.parse('./tutorial.dae')

In [3]:
root = tree.getroot()
root

<Element '{http://www.collada.org/2005/11/COLLADASchema}COLLADA' at 0x000002AC2B1E3F10>

In [4]:
for child in root:
    print(child.tag)

{http://www.collada.org/2005/11/COLLADASchema}asset
{http://www.collada.org/2005/11/COLLADASchema}library_cameras
{http://www.collada.org/2005/11/COLLADASchema}library_lights
{http://www.collada.org/2005/11/COLLADASchema}library_effects
{http://www.collada.org/2005/11/COLLADASchema}library_images
{http://www.collada.org/2005/11/COLLADASchema}library_materials
{http://www.collada.org/2005/11/COLLADASchema}library_geometries
{http://www.collada.org/2005/11/COLLADASchema}library_visual_scenes
{http://www.collada.org/2005/11/COLLADASchema}scene


In [5]:
geometries = [child for child in root[6]]


In [6]:
meshes: Dict[str, dict]
meshes = {}

In [7]:
for i, geom in enumerate(geometries):
    mesh = geom[0]
    name = geom.attrib['name']
    meshes[name] = {}
    for source in mesh:
        print(i, source.tag, source.attrib)
        tag = re.sub(r"\{[^{}]*\}", "", source.tag)
        if tag == 'source':
            attribute = source.attrib['id'].split("-")[1:3]
            attribute = "-".join(attribute)
            meshes[name][attribute] = {}
            for child in source:
                child_tag = re.sub(r"\{[^{}]*\}", "", child.tag)
                meshes[name][attribute][child_tag] = child
        elif tag == 'triangles':
            meshes[name][tag] = {}
            semantics = 0
            for child in source:
                child_tag = re.sub(r"\{[^{}]*\}", "", child.tag)
                if child_tag == 'input':
                    meshes[name][tag][child.attrib['semantic']] = child.attrib['offset']
                    semantics += 1
                elif child_tag == 'p':
                    assert isinstance(child.text, str)
                    meshes[name][tag]['data'] = np.fromstring(child.text, dtype=np.int64, sep=' ').reshape((-1, semantics))
        

0 {http://www.collada.org/2005/11/COLLADASchema}source {'id': 'Cube_002-mesh-positions'}
0 {http://www.collada.org/2005/11/COLLADASchema}source {'id': 'Cube_002-mesh-normals'}
0 {http://www.collada.org/2005/11/COLLADASchema}source {'id': 'Cube_002-mesh-map-0'}
0 {http://www.collada.org/2005/11/COLLADASchema}source {'id': 'Cube_002-mesh-colors-Color', 'name': 'Color'}
0 {http://www.collada.org/2005/11/COLLADASchema}vertices {'id': 'Cube_002-mesh-vertices'}
0 {http://www.collada.org/2005/11/COLLADASchema}triangles {'material': 'Vtxcolor-material', 'count': '44'}
1 {http://www.collada.org/2005/11/COLLADASchema}source {'id': 'Cube_001-mesh-positions'}
1 {http://www.collada.org/2005/11/COLLADASchema}source {'id': 'Cube_001-mesh-normals'}
1 {http://www.collada.org/2005/11/COLLADASchema}source {'id': 'Cube_001-mesh-map-0'}
1 {http://www.collada.org/2005/11/COLLADASchema}vertices {'id': 'Cube_001-mesh-vertices'}
1 {http://www.collada.org/2005/11/COLLADASchema}triangles {'count': '12'}
2 {http:

In [8]:
def reshape_data_into_numpy(meshes: Dict[str, dict], key_to_reshape: str):
    for key in meshes:
        if key_to_reshape in meshes[key].keys():
            meshes[key][key_to_reshape]['float_array'] = np.fromstring(meshes[key][key_to_reshape]['float_array'].text, sep=' ') \
                .reshape((-1, int(meshes[key][key_to_reshape]['technique_common'][0].attrib['stride'])))

In [9]:
to_reshape = ('mesh-positions', 'mesh-normals', 'mesh-map', 'mesh-colors')
list(map(lambda x: reshape_data_into_numpy(meshes, x), to_reshape))

[None, None, None, None]

In [10]:
mapping_names_to_triangle_attribs = {
    'mesh-positions': 'VERTEX',
    'mesh-colors': 'COLOR',
    'mesh-normals': 'NORMAL',
    'mesh-map': 'TEXCOORD'
}
for key in meshes:
    for name in to_reshape:
        if mapping_names_to_triangle_attribs[name] in meshes[key]['triangles']:
            meshes[key]['triangles'][name] = int(meshes[key]['triangles'].pop(mapping_names_to_triangle_attribs[name]))

In [11]:
# class for holding data of each vertex
@dataclass
class Vertex:
    position: List[float]
    normal: List[float]
    texture: List[float]
    colors: Optional[List[float]] = None

# class for triangle to then calculate area
@dataclass
class Triangle:
    vertices: Tuple[Vertex, Vertex, Vertex]

    def cross(self):
        p0 = np.array(self.vertices[0].position, dtype=np.float64)
        p1 = np.array(self.vertices[1].position, dtype=np.float64)
        p2 = np.array(self.vertices[2].position, dtype=np.float64)
        v = p0 - p1
        u = p0 - p2
        return np.cross(v, u)

    def calculate_area(self):
        cross = self.cross()
        self.area = np.linalg.norm(cross) / 2

    def calculate_normals(self):
        # set to get unique values
        normals = {tuple(x.normal) for x in self.vertices}
        if len(normals) == 1:
            self.normal = np.array(self.vertices[0].normal)
        else: 
            cross = self.cross()
            self.normal = cross / np.linalg.norm(cross)
    
    def calculate_midpoints(self):
        positions = [np.array(x.position) for x in self.vertices]
        sum_pos = sum(positions)
        assert isinstance(sum_pos, np.ndarray)
        self.midpoint = sum_pos / len(positions)

    def intersection_triangle(self, light_center: NDArray[np.float64], ray_direction: NDArray[np.float64]) -> bool:

        p0 = np.array(self.vertices[0].position, dtype=np.float64)
        p1 = np.array(self.vertices[1].position, dtype=np.float64)
        p2 = np.array(self.vertices[2].position, dtype=np.float64)

        e1 = p1 - p0
        e2 = p2 - p0
        distance_from_light = np.linalg.norm(ray_direction)
        ray_direction_norm = ray_direction / distance_from_light
        transf_matrix = np.column_stack((-ray_direction_norm, e1, e2))
        x = np.linalg.solve(transf_matrix, light_center - p0)
        t = x[0]
        u = x[1]
        v = x[2]

        condition = (0 <= u) and (0 <= v) and (u + v <= 1) and (0 < t < distance_from_light)
        # if condition:
        #     print(x)

        return condition




In [12]:
def get_idx_from_geometry(geometry: Dict[str, dict], name: str) -> np.ndarray:
    return geometry['triangles']['data'][:, int(geometry['triangles'][name])]

def get_arrays_from_idx(geometry: Dict[str, dict], name: str, idx: np.ndarray) -> np.ndarray:
    df = pd.DataFrame(geometry[name]['float_array'])
    return df.loc[idx, :].to_numpy()


def create_vertex_list(geometry: Dict[str, Dict[str, Any]]) -> List[Vertex]:
    names = [key for key in geometry['triangles'].keys() if key != 'data']

    indexes = list(map(lambda x: get_idx_from_geometry(geometry, x), names))
    arrays = list(map(lambda x: get_arrays_from_idx(geometry, *x), zip(names, indexes)))
    map_names_to_kwargs = {
        'mesh-positions': 'position',
        'mesh-colors': 'colors',
        'mesh-normals': 'normal',
        'mesh-map': 'texture'
    }
    kwargs_df = {map_names_to_kwargs[name]: array.tolist() for name, array in zip(names, arrays)}
    kwargs_df = pd.DataFrame.from_dict(kwargs_df)
    list_triangles = [Vertex(**kwargs) for _, kwargs in kwargs_df.iterrows()]
    return list_triangles
    

In [13]:
vertices = {key: create_vertex_list(meshes[key]) for key in meshes.keys()}

In [14]:
# split into 3-vertices chunks to then put into triangle
def chunk(it: Iterable[Any], size: int):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

triangles = {key: list(map(lambda x: Triangle(x), chunk(vertices[key], 3))) for key in vertices.keys()}

In [15]:
scene_data = [child for child in root[7][0]]

In [16]:
assert isinstance(scene_data[4][0].text, str)
light_matrix = np.fromstring(scene_data[4][0].text, sep=' ').reshape((-1, 4))

In [17]:
light_matrix

array([[-0.2908646 , -0.7711008 ,  0.5663932 ,  5.033271  ],
       [ 0.9551712 , -0.1998834 ,  0.2183912 ,  0.4453149 ],
       [-0.05518906,  0.6045247 ,  0.7946723 ,  4.101171  ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [18]:
light_source_position = light_matrix[:, 3][:3]
light_source_position

array([5.033271 , 0.4453149, 4.101171 ])

In [19]:
light_source_point = Point3D(*light_source_position)

In [20]:
all_triangles: List[Triangle]
all_triangles = []
for key in triangles.keys():
    all_triangles.extend(triangles[key])

In [21]:
# calculate attribs for triangles
for triangle in all_triangles:
    triangle.calculate_area()
    triangle.calculate_midpoints()
    triangle.calculate_normals()

In [22]:
len(all_triangles)

150

In [23]:
from copy import copy


all_triangles_cpy = copy(all_triangles)
visibility = np.ones((len(all_triangles),))
for i, triang in enumerate(all_triangles):
    # print(f"Triangle {i}")
    from_light_to_midpoint_dir = triang.midpoint - light_source_position
    for j, other_triang in enumerate(all_triangles_cpy):
        if i != j and other_triang.intersection_triangle(light_source_position, from_light_to_midpoint_dir):
            visibility[i] = 0
            break
        
    # if i % 5 == 0:
    #     print(visibility)
        
        

In [24]:
visibility

array([0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.])